In [3]:
#linear_regression

# %matplotlib inline
#总结：%matplotlib inline 可以在Ipython编译器里直接使用，功能是可以内嵌绘图，并且可以省略掉plt.show()这一步。

import random
import torch


def synthetic_data(w, b, num_examples):  #@save
    """生成y=Xw+b+噪声"""
    # torch.normal(means, std, out=None)
    # X.shape [1000, 2]
    X = torch.normal(0, 1, (num_examples, len(w))) 
    y = torch.matmul(X, w) + b
    # y偏差处理
    y += torch.normal(0, 0.01, y.shape)
    # reshape(-1, 1) => 转成一列
    # reshape(1, -1) => 转成一行
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

print("features.shape: ", features.shape)
print("labels.shape: ", labels.shape)
print('features:', features[0],'\nlabel:', labels[0])

features.shape:  torch.Size([1000, 2])
labels.shape:  torch.Size([1000, 1])
features: tensor([-0.0269, -0.9423]) 
label: tensor([7.3457])


In [5]:
from numpy.core import setup_common
hex(setup_common.C_API_VERSION)

from d2l import torch as d2l

ModuleNotFoundError: No module named 'numpy'

In [6]:
def data_iter(batch_size, features, labels):
    num_examples = len(features) 
    indices = list(range(num_examples))
    # 这些样本是随机读取的，没有特定的顺序
    random.shuffle(indices)
    # range(start, end, step)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]
    
batch_size = 10

for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break    

tensor([[-2.2827, -0.0829],
        [-0.2207, -0.3191],
        [ 0.2394,  1.8658],
        [-1.6158,  1.4838],
        [-1.0540,  0.4552],
        [-1.5238,  0.3182],
        [-1.8327,  3.5810],
        [-0.3765, -0.8013],
        [ 1.0934, -0.0704],
        [-1.7653,  0.8370]]) 
 tensor([[ -0.0812],
        [  4.8381],
        [ -1.6640],
        [ -4.0686],
        [  0.5451],
        [  0.0726],
        [-11.6456],
        [  6.1656],
        [  6.6150],
        [ -2.1686]])


In [7]:
w = torch.normal(0, 0.01, size=(2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [8]:
def linreg(X, w, b):  #@save
    """线性回归模型"""
    return torch.matmul(X, w) + b

In [9]:
def squared_loss(y_hat, y):  #@save
    """均方损失"""
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

In [10]:
def sgd(params, lr, batch_size):  #@save
    """小批量随机梯度下降"""
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

In [11]:
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss
import sys

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        print(net(X, w, b))
        l = loss(net(X, w, b), y)  # X和y的小批量损失
        # 因为l形状是(batch_size,1)，而不是一个标量。l中的所有元素被加到一起，
        # 并以此计算关于[w,b]的梯度
        l.sum().backward()
        sgd([w, b], lr, batch_size)  # 使用参数的梯度更新参数
    #使用with torch.no_grad():表明当前计算不需要反向传播，使用之后，强制后边的内容不进行计算图的构建    
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

tensor([[ 1.7274e-02],
        [ 7.2219e-03],
        [-3.5919e-03],
        [-7.5692e-03],
        [ 3.1268e-03],
        [ 1.3825e-02],
        [ 4.8983e-03],
        [ 4.2522e-03],
        [-1.1286e-02],
        [-2.4847e-05]], grad_fn=<AddBackward0>)
tensor([[-0.0194],
        [ 0.0531],
        [ 0.0303],
        [ 0.0354],
        [ 0.1257],
        [ 0.0407],
        [ 0.0798],
        [-0.0545],
        [ 0.1032],
        [ 0.0460]], grad_fn=<AddBackward0>)
tensor([[0.3692],
        [0.2748],
        [0.3187],
        [0.0921],
        [0.1752],
        [0.1048],
        [0.1937],
        [0.0042],
        [0.2450],
        [0.2871]], grad_fn=<AddBackward0>)
tensor([[ 0.4954],
        [ 0.1081],
        [ 0.4387],
        [ 0.2234],
        [ 0.2906],
        [ 0.2447],
        [-0.1168],
        [ 0.7412],
        [ 0.3211],
        [-0.0545]], grad_fn=<AddBackward0>)
tensor([[ 0.4011],
        [ 0.9919],
        [-0.3745],
        [ 0.0527],
        [ 0.4426],
        [ 0.457

In [60]:
print(f'w的估计误差: {true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差: {true_b - b}')

w的估计误差: tensor([ 0.0003, -0.0014], grad_fn=<SubBackward0>)
b的估计误差: tensor([0.0002], grad_fn=<RsubBackward1>)
